In [1]:
import glob
import itertools
import json
import os
import pandas
import scipy
from scipy import stats


In [2]:
path = "TutorialData"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

session_map = {}
for filename in dir_list:
    if filename.isdigit():
        exp_paths.append(path + "/" + filename + "/Training/Tutorial/")
        #session_map[filename] = user_id
        files = os.listdir(path + "/" + filename + "/Training/Tutorial/")
        name = files[0]
        index = name.find(".")
        user_id = name[index - 13: index]
        session_map[user_id] = int(filename)

print(exp_paths)

Files and directories in ' TutorialData ' :
['59', '68', '32', '35', '56', '51', '58', '60', '34', '33', '.DS_Store', '45', '44', '43', '36', '31', '30', '37', '39', '46', '41', '70', '49', '40', '78']
['TutorialData/59/Training/Tutorial/', 'TutorialData/68/Training/Tutorial/', 'TutorialData/32/Training/Tutorial/', 'TutorialData/35/Training/Tutorial/', 'TutorialData/56/Training/Tutorial/', 'TutorialData/51/Training/Tutorial/', 'TutorialData/58/Training/Tutorial/', 'TutorialData/60/Training/Tutorial/', 'TutorialData/34/Training/Tutorial/', 'TutorialData/33/Training/Tutorial/', 'TutorialData/45/Training/Tutorial/', 'TutorialData/44/Training/Tutorial/', 'TutorialData/43/Training/Tutorial/', 'TutorialData/36/Training/Tutorial/', 'TutorialData/31/Training/Tutorial/', 'TutorialData/30/Training/Tutorial/', 'TutorialData/37/Training/Tutorial/', 'TutorialData/39/Training/Tutorial/', 'TutorialData/46/Training/Tutorial/', 'TutorialData/41/Training/Tutorial/', 'TutorialData/70/Training/Tutorial/',

In [3]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [4]:
group_map = {}

for i in range (30, 70):
    if(i % 2 == 0):
        group_map[i] = "A"
    else:
        group_map[i] = "B"

In [5]:
responsiveData = []
experimentEvents = []

def parseFrame(frames, user_id):
    frames_list = []
    for frame in frames:
        frameData = {
            "userID": float,
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["userID"] = user_id
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session, user_id):
    sessionData = {
    "userID": float,
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["userID"] = user_id
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [6]:
data = []
frames = []
allsession = []
user_id = 0

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {}
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    user_id = session_map[json_data["obj"]["userID"]]
                    session = parseSession(json_data["obj"]["sessionRecordings"][0], user_id)
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data, user_id)
                    frame = parseFrame(json_data["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


TutorialSession_0_1652906031540.json
TutorialSession_1_1652906031540.json
TutorialSession_2_1652906031540.json
TutorialSession_3_1652906031540.json
TutorialSession_4_1652906031540.json
TutorialSession_5_1652906031540.json
TutorialSession_6_1652906031540.json
TutorialSession_7_1652906031540.json
TutorialSession_8_1652906031540.json
TutorialSession_0_1653068647666.json
TutorialSession_1_1653068647666.json
TutorialSession_2_1653068647666.json
TutorialSession_3_1653068647666.json
TutorialSession_4_1653068647666.json
TutorialSession_5_1653068647666.json
TutorialSession_6_1653068647666.json
TutorialSession_7_1653068647666.json
TutorialSession_8_1653068647666.json
TutorialSession_0_1652733080510.json
TutorialSession_1_1652733080510.json
TutorialSession_2_1652733080510.json
TutorialSession_3_1652733080510.json
TutorialSession_4_1652733080510.json
TutorialSession_5_1652733080510.json
TutorialSession_6_1652733080510.json
TutorialSession_7_1652733080510.json
TutorialSession_8_1652733080510.json
T

In [7]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pandas.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

sessionsDf = sessionsDf.loc[sessionsDf["sessionNumber"] < 8 ]
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,59,256,Select a hologram or button within arm's reach...,1652906043240,1652906057530,14290,0,False,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165..."


In [8]:
sessionsDf["group"] = sessionsDf["userID"] % 2


In [9]:
framesUnpacked = list(itertools.chain(*sessionsDf["frames"]))
framesDf = pandas.DataFrame(framesUnpacked)
print(framesDf.shape)
framesDf["group"] = framesDf["userID"] % 2
framesDf.head(1)

(41091, 14)


,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData,group
0,59,1,1652906031700,160,"{'x': -0.10537330061197281, 'y': 0.00502714980...","{'x': 0.004355150740593672, 'y': 0.06178361549...","{'x': -0.030907930806279182, 'y': 0.0021603247...","{'x': 356.4577941894531, 'y': 0.25001019239425...","{'x': -0.09226678311824799, 'y': 0.00413124728...","{'x': 0.004577492829412222, 'y': 0.04538172483...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","{'x': 0.0, 'y': 0.0, 'z': 0.0}",[],"{'scale': {'x': 0.0, 'y': 0.0, 'z': 0.0}, 'pos...",1


In [10]:
users = framesDf.loc[framesDf["experimentEvents"].str.len() != 0]
print(users.shape)

experimentUnpacked = list(itertools.chain(*users["experimentEvents"]))
experimentDf = pandas.DataFrame(experimentUnpacked)
experimentDf.head(1)
print(experimentDf.shape)
experimentDf.shape

experimentDf["userID"] = users["userID"]

for i in users.index:
    eventDF = pandas.DataFrame(users["experimentEvents"][i])

    for j in eventDF.index:
        time = eventDF["unixTime"][j]
        userID = users["userID"][i]
        mask = experimentDf["unixTime"] == time
        experimentDf.loc[mask, "userID"] = userID

experimentDf.head(1)

(353, 15)
(737, 10)


,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1652906043170,13-34-03-17,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,59.0


In [11]:
startEvents = experimentDf.loc[experimentDf["eventName"] == "start"]
startEvents.head(3)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1652906043170,13-34-03-17,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,59.0
1,1653084254064,15-04-14-06,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,46.0
2,1652895623679,10-40-23-67,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,56.0


In [12]:
sessionsDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group
0,59,256,Select a hologram or button within arm's reach...,1652906043240,1652906057530,14290,0,False,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1
162,46,334,Select a hologram or button within arm's reach...,1653084254144,1653084272984,18840,0,False,"[{'userID': 46, 'frameNum': 1, 'unixTime': 165...",0
36,56,247,Select a hologram or button within arm's reach...,1652895623759,1652895632649,8890,0,False,"[{'userID': 56, 'frameNum': 1, 'unixTime': 165...",0
153,39,247,Select a hologram or button within arm's reach...,1652825201839,1652825215589,13750,0,False,"[{'userID': 39, 'frameNum': 1, 'unixTime': 165...",1
45,45,253,Select a hologram or button within arm's reach...,1653002894394,1653002908465,14071,0,False,"[{'userID': 45, 'frameNum': 1, 'unixTime': 165...",1
...,...,...,...,...,...,...,...,...,...,...
106,44,143,Scroll down by selecting and holding the blue ...,1652992269830,1652992284171,14341,7,False,"[{'userID': 44, 'frameNum': 870, 'unixTime': 1...",0
151,37,115,Scroll down by selecting and holding the blue ...,1652814244479,1652814255912,11433,7,False,"[{'userID': 37, 'frameNum': 987, 'unixTime': 1...",1
34,35,68,Scroll down by selecting and holding the blue ...,1652802697337,1652802704197,6860,7,False,"[{'userID': 35, 'frameNum': 809, 'unixTime': 1...",1
16,68,146,Scroll down by selecting and holding the blue ...,1653068775858,1653068790498,14640,7,False,"[{'userID': 68, 'frameNum': 1261, 'unixTime': ...",0


In [13]:
sessionsDf["Actual_Start_Time"] = sessionsDf["start_time"]

for i in startEvents.index:
    userID = startEvents["userID"][i]
    task_num = startEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_Start_Time"] = startEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time
0,59,256,Select a hologram or button within arm's reach...,1652906043240,1652906057530,14290,0,False,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1,1652906043240


In [14]:
endEvents = experimentDf.loc[experimentDf["eventName"] == "complete"]
endEvents.head(1)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID


In [15]:
sessionsDf["Actual_End_Time"] = sessionsDf["end_time"]

for i in endEvents.index:
    userID = endEvents["userID"][i]
    task_num = endEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_End_Time"] = endEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time,Actual_End_Time
0,59,256,Select a hologram or button within arm's reach...,1652906043240,1652906057530,14290,0,False,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1,1652906043240,1652906057530


In [16]:
end_time_diff = sessionsDf["end_time"].to_numpy() - sessionsDf["Actual_End_Time"].to_numpy()
end_time_diff

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
start_time_diff = sessionsDf["start_time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
start_time_diff

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
        11020,   7121,   8651,   9491,  14212,  16510,  14231,  14041,
         6780,  13970,  14130,  97463,  15578,  12550,  38821,  14290,
         7141,  19043,  11910,  15630,  14231,  14510,   9070,  15079,
        51521,  25211,  16540, 303976,  50155,  41930,  22600,  21781,
        61671,  42811,  19790,  75051,  16990,  35561,  11140,  29701,
        17285,  16820,  17031,  63521,  41930,  36066,  90215,  38791,
        11130,   5860,  12951,   4640,   9871,  11711,   7850,  10050,
        12000,  13950,  11730,   7431,   4435,  11395,   6520,  18410,
         4440,  12791,   6540,  11605,  11060,  12951,  12581,   6618,
         6880,  94072,   4201,   4770,   5600,  25700,   8890,   6040,
         7850,   4991,   7811,   5430,   4009,  57531,   4580,   4870,
      

In [18]:
sessionsDf["Actual_Total_Time"] = sessionsDf["Actual_End_Time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time,Actual_End_Time,Actual_Total_Time
0,59,256,Select a hologram or button within arm's reach...,1652906043240,1652906057530,14290,0,False,"[{'userID': 59, 'frameNum': 1, 'unixTime': 165...",1,1652906043240,1652906057530,14290


In [19]:
start_time_diff = sessionsDf["total_time"].to_numpy() - sessionsDf["Actual_Total_Time"].to_numpy()
start_time_diff

array([      0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,  -11020,   -7121,   -8651,   -9491,
        -14212,  -16510,  -14231,  -14041,   -6780,  -13970,  -14130,
        -97463,  -15578,  -12550,  -38821,  -14290,   -7141,  -19043,
        -11910,  -15630,  -14231,  -14510,   -9070,  -15079,  -51521,
        -25211,  -16540, -303976,  -50155,  -41930,  -22600,  -21781,
        -61671,  -42811,  -19790,  -75051,  -16990,  -35561,  -11140,
        -29701,  -17285,  -16820,  -17031,  -63521,  -41930,  -36066,
        -90215,  -38791,  -11130,   -5860,  -12951,   -4640,   -9871,
        -11711,   -7850,  -10050,  -12000,  -13950,  -11730,   -7431,
         -4435,  -11395,   -6520,  -18410,   -4440,  -12791,   -6540,
        -11605,  -11060,  -12951,  -12581,   -6618,   -6880,  -94072,
         -4201,   -4

In [20]:
#task 0
i = 0
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 0 Time: 12889.909 ± 2480.221
normal task 0 Time: 20692.385 ± 24445.830
Ttest_indResult(statistic=-1.0504030509858817, pvalue=0.3049350247201801)


In [21]:
#Task 1
i = 1
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 1 Time: 72931.182 ± 82197.990
normal task 1 Time: 58333.000 ± 44578.536
Ttest_indResult(statistic=0.5528035079416129, pvalue=0.5859717233112359)


In [22]:
#Task 2
i = 2
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 2 Time: 69763.727 ± 82007.015
normal task 2 Time: 47601.462 ± 24978.472
Ttest_indResult(statistic=0.9281441559163569, pvalue=0.3634047089977893)


In [23]:
#task 3
i = 3
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 3 Time: 22448.727 ± 18766.232
normal task 3 Time: 22731.077 ± 25022.791
Ttest_indResult(statistic=-0.030772802545478314, pvalue=0.9757281997830709)


In [24]:
#task 4
i = 4
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 4 Time: 22792.091 ± 18429.932
normal task 4 Time: 23124.462 ± 26805.422
Ttest_indResult(statistic=-0.03471061968282666, pvalue=0.9726235080696638)


In [25]:
#task 5
i = 5
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 5 Time: 21682.909 ± 10704.030
normal task 5 Time: 19271.846 ± 22129.953
Ttest_indResult(statistic=0.32940783217029146, pvalue=0.7449630767643884)


In [26]:
#task 6
i = 6
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 6 Time: 49261.091 ± 27535.378
normal task 6 Time: 33358.769 ± 18848.182
Ttest_indResult(statistic=1.6728862709511703, pvalue=0.10851202090489732)


In [27]:
#task 7
i = 7
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 7 Time: 83833.818 ± 61275.190
normal task 7 Time: 43564.231 ± 18202.353
Ttest_indResult(statistic=2.2626100398150073, pvalue=0.033875306220829675)


In [28]:
#each task in a loop
for i in range(0,8):
    print(f"TASK {str(i)}:")
    taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["group"] == 1]
    task_n = taskDf.loc[taskDf["group"] == 0]

    taskRAvgTime = task_r['Actual_Total_Time'].mean()
    taskRStd = task_r['Actual_Total_Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Actual_Total_Time'].mean()
    taskNStd = task_n['Actual_Total_Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 0:
responsive task 0 Time: 12889.909 ± 2480.221
normal task 0 Time: 20692.385 ± 24445.830
Ttest_indResult(statistic=-1.0504030509858817, pvalue=0.3049350247201801)
TASK 1:
responsive task 1 Time: 72931.182 ± 82197.990
normal task 1 Time: 58333.000 ± 44578.536
Ttest_indResult(statistic=0.5528035079416129, pvalue=0.5859717233112359)
TASK 2:
responsive task 2 Time: 69763.727 ± 82007.015
normal task 2 Time: 47601.462 ± 24978.472
Ttest_indResult(statistic=0.9281441559163569, pvalue=0.3634047089977893)
TASK 3:
responsive task 3 Time: 22448.727 ± 18766.232
normal task 3 Time: 22731.077 ± 25022.791
Ttest_indResult(statistic=-0.030772802545478314, pvalue=0.9757281997830709)
TASK 4:
responsive task 4 Time: 22792.091 ± 18429.932
normal task 4 Time: 23124.462 ± 26805.422
Ttest_indResult(statistic=-0.03471061968282666, pvalue=0.9726235080696638)
TASK 5:
responsive task 5 Time: 21682.909 ± 10704.030
normal task 5 Time: 19271.846 ± 22129.953
Ttest_indResult(statistic=0.32940783217029146, pvalue=

In [29]:
usersDf = sessionsDf[["userID"]]
usersDf = usersDf.drop_duplicates()
usersDf = usersDf.sort_values("userID")
usersDf

,userID
135,30
126,31
18,32
81,33
72,34
27,35
117,36
144,37
153,39
198,40


In [30]:
allTasksDf = usersDf

In [31]:
for i in range(0,8):
    allTasksDf[f"task{i}"] = i

allTasksDf["all_tasks"] = 0
allTasksDf["group"] = allTasksDf["userID"] % 2
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,all_tasks,group
135,30,0,1,2,3,4,5,6,7,0,0
126,31,0,1,2,3,4,5,6,7,0,1
18,32,0,1,2,3,4,5,6,7,0,0
81,33,0,1,2,3,4,5,6,7,0,1
72,34,0,1,2,3,4,5,6,7,0,0
27,35,0,1,2,3,4,5,6,7,0,1
117,36,0,1,2,3,4,5,6,7,0,0
144,37,0,1,2,3,4,5,6,7,0,1
153,39,0,1,2,3,4,5,6,7,0,1
198,40,0,1,2,3,4,5,6,7,0,0


In [32]:
for i in allTasksDf.index:
    user = allTasksDf["userID"][i]
    userSessions = sessionsDf.loc[sessionsDf["userID"] == user]
    total = 0
    task = userSessions.loc[userSessions["sessionNumber"] == 0]
    allTasksDf.loc[i, "task0"] = task["Actual_Total_Time"].to_numpy()[0]
    total = int(task["Actual_Total_Time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 1]
    allTasksDf.loc[i, "task1"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 2]
    allTasksDf.loc[i, "task2"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 3]
    allTasksDf.loc[i, "task3"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 4]
    allTasksDf.loc[i, "task4"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 5]
    allTasksDf.loc[i, "task5"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 6]
    allTasksDf.loc[i, "task6"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 7]
    allTasksDf.loc[i, "task7"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"].to_numpy()[0])
    allTasksDf.loc[i, "all_tasks"] = total
    if(task["isResponsive"].to_numpy()[0] == True):
        allTasksDf.loc[i, "group"] = "B"

#allTasksDf = allTasksDf.drop(columns=["task{i}"])

In [33]:
allTasksDf = allTasksDf.sort_values("all_tasks")
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,all_tasks,group
171,41,6960,23861,22660,10620,9190,9390,17870,24110,124661,1
18,32,10835,27920,28030,15051,8509,10860,22420,26349,149974,0
63,60,6921,26811,26310,11390,9450,8900,22370,38761,150913,0
27,35,14880,26129,15770,11260,17040,13501,26387,30171,155138,1
36,56,8890,26250,21710,11550,13894,9770,22501,44420,158985,0
207,78,8460,25591,29581,16501,15440,17071,25011,30051,167706,0
99,44,11749,37011,36331,15000,7960,7859,21490,32081,169481,0
144,37,14036,49661,39991,10470,15820,17480,22521,26270,196249,1
72,34,13850,36550,35371,18212,10520,10640,36881,42891,204915,0
117,36,15451,37311,31811,15640,13040,23890,37331,31177,205651,0


In [34]:
groupA = allTasksDf.loc[allTasksDf["group"] == 0]
groupB = allTasksDf.loc[allTasksDf["group"] == 1]

groupATime = groupA['all_tasks'].mean()
groupAStd = groupA['all_tasks'].std()
print(f"group A {str(i)} Time: {groupATime:.3f} ± {groupAStd:.3f}")

groupBTime = groupB['all_tasks'].mean()
groupBStd = groupB['all_tasks'].std()
print(f"group B {str(i)} Time: {groupBTime:.3f} ± {groupBStd:.3f}")

print(scipy.stats.ttest_ind(groupB['all_tasks'], groupA['all_tasks']))

group A 207 Time: 268677.231 ± 147217.942
group B 207 Time: 347081.000 ± 206176.083
Ttest_indResult(statistic=1.0655295920152588, pvalue=0.2987345593438125)


In [35]:
sessionsDf.loc[(sessionsDf["userID"] == 30) & (sessionsDf["sessionNumber"] == 0)]

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time,Actual_End_Time,Actual_Total_Time
135,30,2009,Select a hologram or button within arm's reach...,1652717155289,1652717170466,15177,0,False,"[{'userID': 30, 'frameNum': 1, 'unixTime': 165...",0,1652717155289,1652717170466,15177
